In [ ]:
import numpy as np
from numpy import array as arr

import pandas
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib as mpl
# mpl.use("pdf")
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
mpl.rcParams["font.serif"] = "CMU serif"
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.rm"] = "CMU serif"
mpl.rcParams["mathtext.it"] = "CMU serif:italic"
mpl.rcParams["mathtext.bf"] = "CMU serif:bold"
mpl.rcParams["font.family"] = "serif"

# import pylandau
from scipy.optimize import curve_fit
import scipy
import uncertainties as unc
from uncertainties import unumpy as unp
from uncertainties.unumpy import uarray as uarr
from uncertainties.unumpy import nominal_values as val
from uncertainties.unumpy import std_devs as dev
from uncertainties import ufloat as uf
# import ROOT

# import my plotting stuff
import sys
sys.path.append('./PythonHelpers/')
# import PlotLib.Stats as Stats
import PlotLib.Hist1D as Hist1D
import PlotLib.Hist2D as Hist2D
import PlotLib.Plotting as Plot
from CSVimporter.importer import load_run
from CSVimporter.importer import dict_to_arr

import time

def fitfunc_gauss(x, A, mu, sigma):
    return A * np.exp(-0.5*((x-mu)/sigma)**2)

import ROOT

In [ ]:
nEvents = 4*50000
# nEvents = 4*200
runIDs = arr([184,190,192])
runIDs = arr([190])
thr = 220
MTP = 6
filepaths = {
    190:f"/home/jona/DESY/analysis_TB/output/csv/190_thr{thr:n}_trkPlanes{MTP:n}.csv",
}

runs = {}
for runID in runIDs:
    
    runs[runID] = load_run(runID, nEvents*4, filepaths[runID])
    # runs[runID]["data"] has the following structure:
    # [ event , pixel, entry ] where entry is the index given by the map M
    
    runs[runID]["nEvts"] = min(runs[runID]["nEvts"],nEvents)
    runs[runID]["threshold"] = thr
    
M = runs[runIDs[0]]["M"]

def findMax(func, par, x0=10):
    def f(x, par):
        return -1 * func(x, par)
    f2 = np.vectorize(f, excluded=[1])
    return scipy.optimize.fmin(f2, x0, args=(par,), disp=False)[0]

print(M)
print(runs[runID]["data"].shape)

In [ ]:
# mask = runs[runID]["data"][:,:,M["Charge"]] > 1
# mask = np.logical_and(mask, runs[runID]["data"][:,:,M["PixInCluster"]] == 1.)
entries = arr(runs[runID]["data"][:,0,M["Charge"]].flatten())
entries = entries[:20000]

binN = int(np.sqrt(len(entries))*0.2)
print(binN)
binRange = arr([0,400])
hist, bins = np.histogram(entries, bins=binN, range=binRange)

fit_range = arr([0, 150])

TH1D = ROOT.TH1D("TH1D", "TH1D", binN, binRange[0], binRange[1])
for i in range(binN):
    TH1D.SetBinContent(i+1, hist[i])
    TH1D.SetBinError(i+1, np.sqrt(hist[i]))

# Gauss
if False:
    TF1_gauss = ROOT.TF1("TF1_gauss", "[0]*TMath::Gaus(x, [1], [2])", 0, 210)
    
    TF1_gauss.SetParName(0, "A")
    TF1_gauss.SetParameter(0, 600)
    TF1_gauss.SetParLimits(0, 0, 1E6)
    
    TF1_gauss.SetParName(1, "mu")
    TF1_gauss.SetParameter(1, 80)
    TF1_gauss.SetParLimits(1, 0, 200)
    
    TF1_gauss.SetParName(2, "sigma")
    TF1_gauss.SetParameter(2, 30)
    TF1_gauss.SetParLimits(2, 0, 200)
    
    TH1D.Fit(TF1_gauss, "R+")

if False:
    TF1_expModGauss = ROOT.TF1("TF1_expModGauss", "[0]*[1]/2*TMath::Exp([3]/2*(2*[1]+[3]*[2]*[2]-2*x))*TMath::Erfc(([1]+[3]*[2]*[2]-x)/TMath::Sqrt(2)/[2])", 0, 210)
    TF1_expModGauss.SetParName(0, "A")
    TF1_expModGauss.SetParameter(0, 2E3)
    TF1_expModGauss.SetParLimits(0, 0, 1E6)

    TF1_expModGauss.SetParName(1, "mu")
    TF1_expModGauss.SetParameter(1, 100)
    TF1_expModGauss.SetParLimits(1, 0, 200)

    TF1_expModGauss.SetParName(2, "sigma")
    TF1_expModGauss.SetParameter(2, 30)
    TF1_expModGauss.SetParLimits(2, 0, 100)

    TF1_expModGauss.SetParName(3, "lambda")
    TF1_expModGauss.SetParameter(3, 0.01)
    TF1_expModGauss.SetParLimits(3, 0, 0.015)

    TH1D.Fit(TF1_expModGauss, "LR+")

if False:
    TF1_Gumbel = ROOT.TF1("TF1_Gumbel", "[0]/[2]*TMath::Exp(-(x-[1])/[2]-TMath::Exp(-(x-[1])/[2]))", *fit_range)
    TF1_Gumbel.SetParName(0, "A")
    TF1_Gumbel.SetParameter(0, 3E3)
    TF1_Gumbel.SetParLimits(0, 0, 1E6)
    
    TF1_Gumbel.SetParName(1, "mu")
    TF1_Gumbel.SetParameter(1, 50)
    TF1_Gumbel.SetParLimits(1, 0, 200)
    
    TF1_Gumbel.SetParName(2, "beta")
    TF1_Gumbel.SetParameter(2, 10)
    TF1_Gumbel.SetParLimits(2, 0, 50)
    
    TF1_Gumbel.SetLineColor(ROOT.kGreen)
    TH1D.Fit(TF1_Gumbel, "LR+")

if False:
    TF1_GenGauss = ROOT.TF1("TF1_GenGauss", "[0]*TMath::Exp(-( TMath::Power(TMath::Abs(x-[1])/[2], [3])))", *fit_range)

    TF1_GenGauss.SetParName(0, "A")
    TF1_GenGauss.SetParameter(0, 3E2)
    TF1_GenGauss.SetParLimits(0, 0, 1E6)
    
    TF1_GenGauss.SetParName(1, "mu")
    TF1_GenGauss.SetParameter(1, 70)
    TF1_GenGauss.SetParLimits(1, 0, 200)
    
    TF1_GenGauss.SetParName(2, "sigma")
    TF1_GenGauss.SetParameter(2, 30)
    TF1_GenGauss.SetParLimits(2, 0, 100)
    
    TF1_GenGauss.SetParName(3, "beta")
    TF1_GenGauss.SetParameter(3, 2)
    TF1_GenGauss.SetParLimits(3, 0, 10)
    
    TF1_GenGauss.SetLineColor(ROOT.kRed)
    TH1D.Fit(TF1_GenGauss, "LR+")
    
if False:
    TF1_ModGenGauss = ROOT.TF1("TF1_ModGenGauss", "[0]*TMath::Exp(-( TMath::Power(TMath::Abs(x-[1])/[2], [3])))*(1 + TMath::Erfc([4]*(x-[1])/[2])/TMath::Sqrt(2))", *fit_range)
    
    TF1_ModGenGauss.SetParName(0, "A")
    TF1_ModGenGauss.SetParameter(0, 3E2)
    TF1_ModGenGauss.SetParLimits(0, 0, 1E6)
    
    TF1_ModGenGauss.SetParName(1, "mu")
    TF1_ModGenGauss.SetParameter(1, 70)
    TF1_ModGenGauss.SetParLimits(1, 0, 200)
    
    TF1_ModGenGauss.SetParName(2, "sigma")
    TF1_ModGenGauss.SetParameter(2, 30)
    TF1_ModGenGauss.SetParLimits(2, 0, 100)
    
    TF1_ModGenGauss.SetParName(3, "beta")
    TF1_ModGenGauss.FixParameter(3, 2.3)
    # TF1_ModGenGauss.SetParameter(3, 2)
    # TF1_ModGenGauss.SetParLimits(3, 0, 10)
    
    TF1_ModGenGauss.SetParName(4, "alpha")
    TF1_ModGenGauss.SetParameter(4, 0.1)
    TF1_ModGenGauss.SetParLimits(4, 0, 10)
    
    TF1_ModGenGauss.SetLineColor(ROOT.kOrange)
    TH1D.Fit(TF1_ModGenGauss, "LR+")
    
if False:
    TF1_Rice = ROOT.TF1("TF1_Rice", "[0]*x/[2]/[2] *TMath::Exp(-0.5*(x*x+[1]*[1])/[2]/[2])*TMath::BesselI0(x*[1]/[2]/[2])", *fit_range)
    
    TF1_Rice.SetParName(0, "A")
    TF1_Rice.SetParameter(0, 3E2)
    TF1_Rice.SetParLimits(0, 0, 1E6)
    
    TF1_Rice.SetParName(1, "mu")
    TF1_Rice.SetParameter(1, 70)
    TF1_Rice.SetParLimits(1, 0, 200)
    
    TF1_Rice.SetParName(2, "sigma")
    TF1_Rice.SetParameter(2, 30)
    TF1_Rice.SetParLimits(2, 0, 100)
    
    TF1_Rice.SetLineColor(ROOT.kGreen)  
    TH1D.Fit(TF1_Rice, "LR+")
    
if False:
    fLandau = ROOT.TF1("landau", "[0]*TMath::Landau(x, [1], [2])", fit_range[0], fit_range[1]) 
        # multiplying by factor [0] is possible because
        # Integral([0]*f1(x-y)*f2(y)dy) = A*Integral(f1(x-y)*f2(y)).
    fGauss = ROOT.TF1("gauss", "TMath::Gaus(x, [0], [1], true)", fit_range[0], fit_range[1])

    # Convolute
    rConv = ROOT.TF1Convolution(fLandau, fGauss)
    rConv.SetNofPointsFFT(1000)

    # Setup fit function
    TF1_LanGau = ROOT.TF1("FitFunc", rConv, fit_range[0], fit_range[1], rConv.GetNpar())

    # Estimate initial parameters, specific to a LanGau fit
    h_integral = TH1D.Integral(0, TH1D.GetNbinsX(), "width")
    h_width = fit_range[1] - fit_range[0]

    TF1_LanGau.SetParName(0,"Scale")
    TF1_LanGau.SetParameter(0, h_integral)
    TF1_LanGau.SetParLimits(0, 0, 100000*h_integral)

    TF1_LanGau.SetParName(1,"MPV")
    TF1_LanGau.SetParameter(1,
        TH1D.GetBinCenter(TH1D.GetMaximumBin()))
    TF1_LanGau.SetParLimits(1,
        TH1D.GetBinLowEdge(1),
        TH1D.GetBinLowEdge(TH1D.GetNbinsX()) + TH1D.GetBinWidth(TH1D.GetNbinsX()))

    TF1_LanGau.SetParName(2,"Width")
    TF1_LanGau.SetParameter(2, 0.03*h_width)
    TF1_LanGau.SetParLimits(2,0.005*h_width,0.4*h_width)

    TF1_LanGau.FixParameter(3,0)

    TF1_LanGau.SetParName(4,"Sigma")
    TF1_LanGau.SetParameter(4, 0.03*h_width)
    TF1_LanGau.SetParLimits(4, 0.005*h_width, 0.4*h_width)

    TF1_LanGau.SetLineColor(ROOT.kBlue)
    TH1D.Fit(TF1_LanGau, "LR+")

# Two Gauss
if False:
    TF1_2Gauss = ROOT.TF1("TF1_2Gauss", "[0]*TMath::Gaus(x, [1], [2]) + [3]*TMath::Gaus(x, [4], [5])", *fit_range)
    
    TF1_2Gauss.SetParameters(
        600, 80, 30,
        100, 120, 30
    )
    
    TF1_2Gauss.SetParLimits(3, 0, 120)
    TF1_2Gauss.SetParLimits(4, 30, 70)
    
    TH1D.Fit(TF1_2Gauss, "R+")

TCanvas = ROOT.TCanvas("TCanvas", "TCanvas", 800, 600)
# TF1_gauss.Draw()

TH1D.Draw("same")
TCanvas.SaveAs("test.pdf")


In [ ]:
FileName = "inpix/Hitmap"

runID = 190
thr = 180

zRange = arr([5,25])
binN = arr([14,10])
binRange = arr([[-0.5,1.5], [-0.5,1.5]])
cols = [M["TrkCol"], M["TrkRow"]]

hist = Hist2D.Hist_2D(binN, binRange)

mask = runs[runID]["data"][:,:,M["Charge"]] > thr
# mask = np.logical_and(mask, runs[runID]["data"][:,:,M["ClstSize"]] == 1.)
mask = np.logical_and(mask, runs[runID]["data"][:,:,M["Charge"]] == arr([np.max(runs[runID]["data"][:,:,M["Charge"]], axis=1) for i in range(4)]).T)

xs = runs[runID]["data"][:,:,cols[0]].flatten()[mask.flatten()]
ys = runs[runID]["data"][:,:,cols[1]].flatten()[mask.flatten()]

hist.fill(xs, ys)

fig, ax = Plot.create_fig(figsize=(5,7))
label = f"Entries ({int(hist.getIntegral(False)/1000)}k tot.)"
hist.draw(ax, aspect=5/7, overflow=False, cbar_label=label)

ax.set_xticks(arr([-0.5, 0, 0.5, 1, 1.5]))
ax.set_yticks(arr([-0.5, 0, 0.5, 1, 1.5]))
Plot.finalize(runs[runID], fig, ax, 
    title=r"Hitmap",
    xlabel=r"pixel column",
    ylabel= r"pixel row",
    param_dict={"campaign":True, "sample":True, "ER1Param":True, "recoParam":True, "fontsize":8, "thr":thr, "edgeCut":None, "minTrkPlanes":MTP},
    grid=False, legend_loc="upper right")
Plot.savefig(fig, FileName)

In [ ]:
FileName = "inpix/ToT"
runID = 190
thr = 180

binN = arr([14,10])
# binN = arr([21,15])
# binN = arr([6,6])
binRange = arr([[-0.5,1.5], [-0.5,1.5]])
cols = [M["TrkCol"], M["TrkRow"], M["ToT"]]

mask = runs[runID]["data"][:,:,M["Charge"]] > thr
# mask = np.logical_and(mask, runs[runID]["data"][:,:,M["Risetime"]] > 0)
entries = arr([runs[runID]["data"][:,:,col].flatten()[mask.flatten()] for col in cols])

hist = Hist2D.Plot_2D(binN, binRange, statistic="LanGau-MPV", inBin_binNdelta=15)

hist.fillData(entries[0], entries[1], entries[2])

fig, ax = Plot.create_fig(figsize=(5,7))
label = f"ToT (MPV of LanGau Fit) / ns"
# hist.draw(ax, aspect=5/7, overflow=False, cbar_label=label, vmin=100, vmax=400)
hist.draw(ax, aspect=5/7, overflow=False, cbar_label=label)

Plot.finalize(runs[runID], fig, ax, 
    title="Time over Threshold",
    xlabel=r"pixel column / pitch ($35\,\mu$m)",
    ylabel= r"pixel row / pitch ($25\,\mu$m)",
    param_dict={"campaign":True, "sample":True, "ER1Param":True, "recoParam":True, "fontsize":8, "thr":thr, "edgeCut":None, "minTrkPlanes":MTP},
    grid=False, legend_loc="upper right")
Plot.savefig(fig, FileName)

In [ ]:
FileName = "inpix/Charge"
runID = 190
thr = 220

binN = arr([14,10])
binRange = arr([[-0.5,1.5], [-0.5,1.5]])
cols = [M["TrkCol"], M["TrkRow"], M["Charge"]]

mask = runs[runID]["data"][:,:,M["Charge"]] > thr
# mask = np.logical_and(mask, runs[runID]["data"][:,:,M["PixInCluster"]] == 1.)
entries = arr([runs[runID]["data"][:,:,col].flatten()[mask.flatten()] for col in cols])

# hist = Hist2D.Plot_2D(binN, binRange, statistic="LanGau-MPV", inBin_binNdelta=5)
hist = Hist2D.Plot_2D(binN, binRange, sFalse
fig, ax = Plot.create_fig(figsize=(5,7))
label = f"Charge (PMV of LanGau Fit) / ns"
# hist.draw(ax, aspect=5/7, overflow=False, cbar_label=label, vmin=100, vmax=400)
hist.draw(ax, aspect=5/7, overflow=False, cbar_label=label)

Plot.finalize(runs[runID], fig, ax, 
    title="Charge",
    xlabel=r"pixel column / pitch ($35\,\mu$m)",
    ylabel= r"pixel row / pitch ($25\,\mu$m)",
    param_dict={"campaign":True, "sample":True, "ER1Param":True, "recoParam":True, "fontsize":8, "thr":thr, "edgeCut":None, "minTrkPlanes":MTP},
    grid=False, legend_loc="upper right")
Plot.savefig(fig, FileName)

In [ ]:
FileName = "inpix/Risetime"
runID = 190
thr = 180

binN = arr([14,10])
# binN = arr([21,15])
binRange = arr([[-0.5,1.5], [-0.5,1.5]])
cols = [M["TrkCol"], M["TrkRow"], M["Risetime"]]
Bootstrap_N = 10

mask = runs[runID]["data"][:,:,M["Charge"]] > thr
mask = np.logical_and(mask, runs[runID]["data"][:,:,M["ClstSize"]] == 1)
entries = arr([runs[runID]["data"][:,:,col].flatten()[mask.flatten()] for col in cols])

hist = Hist2D.Plot_2D(binN, binRange, statistic="MedianBootstrap", Bootstrap_N=Bootstrap_N)
hist.fillData(entries[0], entries[1], entries[2])

fig, ax = Plot.create_fig(figsize=(5,7))
label = f"Risetime (20%-80%, bootstrapped Median) / ns"
hist.draw(ax, aspect=5/7, overflow=False, cbar_label=label)
Plot.finalize(runs[runID], fig, ax, 
    title="Risetime",
    xlabel=r"pixel column / pitch ($35\,\mu$m)",
    ylabel= r"pixel row / pitch ($25\,\mu$m)",
    param_dict={"campaign":True, "sample":True, "ER1Param":True, "recoParam":True, "fontsize":8, "thr":thr, "edgeCut":None, "minTrkPlanes":MTP},
    grid=False, legend_loc="upper right")
Plot.savefig(fig, FileName)

In [ ]:
FileName = "inpix/ToA"
runID = 190
thr = 220

binN = arr([14,10])
binRange = arr([[-0.5,1.5], [-0.5,1.5]])
cols = [M["TrkCol"], M["TrkRow"], M["ToA"]]
Bootstrap_N = 1000

mask = runs[runID]["data"][:,:,M["Charge"]] > thr
entries = arr([runs[runID]["data"][:,:,col].flatten()[mask.flatten()] for col in cols])

hist = Hist2D.Plot_2D(binN, binRange, statistic="MedianBootstrap", Bootstrap_N=Bootstrap_N)
hist.fillData(entries[0], entries[1], entries[2])

fig, ax = Plot.create_fig(figsize=(5,7))
label = f"Risetime (20%-80%, bootstrapped Median) / ns"
hist.draw(ax, aspect=5/7, overflow=False, cbar_label=label)
Plot.finalize(runs[runID], fig, ax, 
    title="ToA", 
    xlabel=r"pixel column / pitch ($35\,\mu$m)",
    ylabel= r"pixel row / pitch ($25\,\mu$m)",
    param_dict={"campaign":True, "sample":True, "ER1Param":True, "recoParam":True, "fontsize":8, "thr":thr, "edgeCut":None, "minTrkPlanes":MTP},
    grid=False, legend_loc="upper right")

In [ ]:
# draw hists created in previous cell
cm = mpl.colormaps["viridis"]
%matplotlib inline
# %matplotlib qt

Methods = arr(["MedianBootstrap"])
runIDs = arr([190])

slice_row = 2
slice_col = 3

RegionBinN = 30
RegionBinRange = arr([0,35])
RegionColorMap = {0:"C1", 1:"C2", 2:"C3", 3:"C4", 4:"C6"}
RegionNames = {0:"Electrode", 1:"Leftmost", 2:"Toroid around Electrode", 3:"Center", 4:"Rightmost"}

draw_sliceX = False
draw_sliceY = False
draw_regions = False

for runID in runIDs:
    print("RunID:", runID)
    for Method in Methods:
        def draw_rectangle(ax, extent, color, zorder=10000):
            x = arr([extent[0,0], extent[0,1], extent[0,1], extent[0,0], extent[0,0]])
            y = arr([extent[1,0], extent[1,0], extent[1,1], extent[1,1], extent[1,0]])
            ax.plot(x,y, color=color, zorder=zorder, ls=":")
        
        hist = runs[runID]["hist_"+Method]
        
        fig, ax = Hist.create_fig(1,1, figsize=[4.4,5])
        
        if False:
            Color_hor = "C3"
            Color_vert = "C1"
            extent_hor = arr([[binRange[0][0], binRange[0][1]], [hist.binsY[slice_row], hist.binsY[slice_row+1]]])
            draw_rectangle(ax, extent_hor, Color_hor)
            extent_vert = arr([[hist.binsX[slice_col], hist.binsX[slice_col+1]], [binRange[1][0], binRange[1][1]]])
            draw_rectangle(ax, extent_vert, Color_vert)
        if False:
            def hatch_fill(x, y, ax, color="C0", hatch="///", alpha=1, zorder=1000, linewidth=1):
                ax.fill_between(x, y[0], y[1], facecolor="none", hatch=hatch, edgecolor=color, linewidth=0.0, alpha=alpha, zorder=zorder)
            def get_extent_X(bin):
                return arr([hist.binsX[bin], hist.binsX[bin+1]])
            def get_extent_Y(bin):
                return arr([hist.binsY[bin], hist.binsY[bin+1]])
            for i in range(RegionMap.shape[0]):
                for j in range(RegionMap.shape[1]):
                    hatch_fill(get_extent_X(i), get_extent_Y(j), ax, color=RegionColorMap[RegionMap[i,j]], hatch="XX", alpha=1)
            
        im, ax_cbar = hist.draw(ax, extent=binRange.flatten(), cbar_label="Risetime [ns]", overflow=True, vmin=zRange[0], vmax=zRange[1])
        
        ax.set_aspect(25/35)
        ax.set_xticks([0,1])
        ax.set_yticks([0,1])
        
        if Method == "MedianBootstrap":
            Title = " (bin median)"
        elif Method == "Mean":
            Title = " (bin mean)"
        elif Method == "MPV":
            Title = " (bin MPV)"
        else:
            Title = " ("+Method+")"
        Hist.finalize(runs[runID], fig, ax, title="Inpixel risetime"+Title, xlabel="x [pix]", ylabel="y [pix]")
        
        left, bottom, widht, height = ax.get_position().bounds
        bottom = 1 - height
        ax.set_position([left, bottom, widht, height])
        
        fig.savefig("output/Inpixel/"+str(runID)+"_"+Method+".pdf")
        fig.savefig("output/Inpixel/"+str(runID)+"_"+Method+".png", dpi=400)
        
        for i in range(RegionMap.shape[0]):
            for j in range(RegionMap.shape[1]):
                RegionEntries[RegionMap[i,j]] = np.append(RegionEntries[RegionMap[i,j]], hist.getBinData(i,j))
                
        if draw_sliceX:
            fig, ax = Hist.create_fig(1,1, figsize=[5,4])

            slice_bins = hist.binsX
            slice_hist = hist.val[1:-1,slice_row+1]
            slice_hist_e = np.sqrt(hist.err[1:-1,slice_row+1])
            slice_binWidth = hist.getBinWidthX()
            slice_xVal = slice_bins[:-1] + slice_binWidth/2
            
            # ax.axhline(0, color="gray", linestyle="--", lw=1)
            ax.errorbar(slice_xVal, slice_hist, xerr=slice_binWidth/2, yerr=slice_hist_e, color=Color_hor, linestyle="", capsize=3)
            ax.set_xlim(hist.getBinRangeX())
            
            Hist.finalize(runs[runID], fig, ax, title="Risetime - slice hor."+Title, xlabel="x [pix]", ylabel="Risetime [ns]")
            ax.grid(False)
            fig.savefig("output/Inpixel/"+str(runID)+"_"+Method+"_slice_hor"+".pdf")
            fig.savefig("output/Inpixel/"+str(runID)+"_"+Method+"_slice_hor"+".png", dpi=400)    
        if draw_sliceY:
            fig, ax = Hist.create_fig(1,1, figsize=[5,4])
            
            slice_bins = hist.binsY
            slice_hist = hist.val[1:-1,slice_row+1]
            slice_hist = hist.val[slice_col+1,1:-1]
            slice_hist_e = np.sqrt(hist.err[slice_col+1,1:-1])
            slice_binWidth = hist.getBinWidthY()
            slice_xVal = slice_bins[:-1] + slice_binWidth/2
            
            ax.errorbar(slice_xVal, slice_hist, xerr=slice_binWidth/2, yerr=slice_hist_e, color=Color_vert, linestyle="", capsize=3)
            ax.set_xlim(hist.getBinRangeY())
            
            Hist.finalize(runs[runID], fig, ax, title="Risetime - slice vert."+Title, xlabel="y [pix]", ylabel="Risetime [ns]", param_narrow=True, param_fontsize=6)
            ax.grid(False)
            fig.savefig("output/Inpixel/"+str(runID)+"_"+Method+"_slice_vert"+".pdf")            
            fig.savefig("output/Inpixel/"+str(runID)+"_"+Method+"_slice_vert"+".png", dpi=400)            
        
        if draw_regions:
            fig1, ax1 = Hist.create_fig(1,1, figsize=[5,4])
            fig2, ax2 = Hist.create_fig(1,1, figsize=[5,4])
            
            RegionHist = np.empty(RegionBinN, dtype=object)
            RegionBins = np.linspace(RegionBinRange[0], RegionBinRange[1], RegionBinN+1)
                
            for i in range(len(RegionEntries)):
                RegionHist[i], _ = np.histogram(RegionEntries[i][1:], bins=RegionBinN, range=RegionBinRange)
                RegionHist[i] = RegionHist[i]/np.sum(RegionHist[i])
            
            baseline = np.zeros(RegionBinN)
            sum = 0
            for i in range(len(RegionEntries)):
                sum += np.sum(RegionHist[i])
            ax2_lines = np.empty(len(RegionEntries),dtype=object)
            ax2_fills = np.empty(len(RegionEntries),dtype=object)
            for i in range(len(RegionEntries)):
                ax1.stairs(RegionHist[i]/sum+baseline, RegionBins, fill=True, color=RegionColorMap[i], baseline=baseline, zorder=1000+i, label=RegionNames[i])
                ax1.stairs(RegionHist[i]/sum+baseline, RegionBins, fill=False, color="black", lw=.5, baseline=baseline, zorder=2000+i)
                baseline += RegionHist[i]/sum

                ax2_lines[i] = ax2.stairs(RegionHist[i], RegionBins, fill=True, color=RegionColorMap[i], alpha=0.2, zorder=1000+i)
                ax2_fills[i] = ax2.stairs(RegionHist[i], RegionBins, fill=False, color=RegionColorMap[i], zorder=2000+i)
            
            ax2.legend([(ax2_fills[i], ax2_lines[i]) for i in range(len(RegionEntries))], [RegionNames[i] for i in range(len(RegionEntries))], frameon=False, prop={'size': 8})
    
            Hist.finalize(runs[runID], fig1, ax1, title="Risetime", xlabel="risetime [ns]", ylabel="Counts (each segment normalized to 1/5)")
            Hist.finalize(runs[runID], fig2, ax2, title="Risetime", xlabel="risetime [ns]", ylabel="Relative Counts")

            fig1.savefig("output/Inpixel/"+str(runID)+"_Risetime_stacked.pdf")
            fig1.savefig("output/Inpixel/"+str(runID)+"_Risetime_stacked.png", dpi=400)
            fig2.savefig("output/Inpixel/"+str(runID)+"_Risetime.pdf")
            fig2.savefig("output/Inpixel/"+str(runID)+"_Risetime.png", dpi=400)

In [ ]:
FileName = "Risetime_BinWise"
binRange = [0,35]
binN = 20
bins = 14

fig, axs = Hist.create_fig(2,7, [8,14])
Text = np.empty(bins, dtype=object)
handles = np.empty(bins, dtype=object)

print(len(hist.data[:,slice_row+1]))

for i_bin in range(bins):
    bin_entries = hist.data[i_bin+1, slice_row+1]
    bin_entries.sort()
    top90 = int(len(bin_entries)*0.95)
    bin_entries = bin_entries[:top90]

    bin_hist, bin_bins = np.histogram(bin_entries, bins=binN, range=binRange)

    axs[i_bin].stairs(bin_hist, bin_bins, fill=True, color="black", alpha=0.5)
    axs[i_bin].stairs(bin_hist, bin_bins, fill=False, color="black")

    mu = bin_entries.mean()
    sig = bin_entries.std()
    axs[i_bin].axvline(mu, color="red", label="Mean (trunc. @ 95%)")
    axs[i_bin].axvline(mu+sig, ls=":", color="red", label="Std (trunc. @ 95%)")
    axs[i_bin].axvline(mu-sig, ls=":", color="red")
    axs[i_bin].set_xlim(binRange)

    Text = "{:.2f} < x [pix] < {:.2f}".format(hist.binsX[i_bin], hist.binsX[i_bin+1]) + "\n" + "{:.2f} < y [pix] < {:.2f}".format(hist.binsY[slice_row], hist.binsY[slice_row+1])
    # handles = mpl.patches.Patch(color='white', label=Text[i_bin])
    patch = mpl.patches.Patch(color='white', label=Text)
    handles[i_bin], labels = axs[i_bin].get_legend_handles_labels()
    handles[i_bin].append(patch)


Hist.finalize(runs[runID], fig, ax, title="Risetime", xlabel="risetime [ns]", ylabel="Entries", param_narrow=True, param_fontsize=6)
for i_bin in range(bins):
    axs[i_bin].legend(handles=handles[i_bin], loc="upper right", fontsize=6)
for i in range(2):
    axs[-i-1].set_xlabel("Risetime [ns]", loc="right")
for i in range(7):
    axs[2*i].set_ylabel("Entries", loc="top")
fig.savefig("output/Inpixel/"+str(runID)+"_slice"+str(slice_row+1)+"_binwise.pdf")
fig.savefig("output/Inpixel/"+str(runID)+"_slice"+str(slice_row+1)+"_binwise.png", dpi=400)

In [ ]:
# FileName = "Risetime_MPV"
# runID = 192
# zRange = [5,175]
runIDs = arr([184,190])
zRange = [5,40]
binN = arr([14,10])
cols = [M["TrkCol"], M["TrkRow"], M["Risetime"]]
cm = mpl.colormaps["viridis"]
%matplotlib inline

slice_row = 2


def draw_colorbar(ax, label=None, width="5%", height="100%", **kwargs):
    cbar_ax = cbar_ax = inset_axes(ax, width=width, height=height, loc = 'lower left',
                   bbox_to_anchor = (1.02, 0., 1, 1), bbox_transform = ax.transAxes,
                   borderpad = 0)
    cbar = plt.colorbar(im, cax=cbar_ax, **kwargs)
    cbar.set_label(label, loc="top")
    return cbar_ax, cbar

for runID in runIDs:
    mask = runs[runID]["data"][:,:,M["Amplitude"]] > 0
    mask = np.logical_and(mask, runs[runID]["data"][:,:,M["Risetime"]] > 0)
    entries = arr([runs[runID]["data"][:,:,col].flatten()[mask.flatten()] for col in cols])
    binRange = arr([[-0.5,1.5], [-0.5,1.5]])

    hist = Plot_2D(binN, binRange, mode="MPV")
    hist.fill_entries(entries[0], entries[1], entries[2], MPV_binRange=[0,50], MPV_binN=100)

    fig, ax = Hist.create_fig(1,1, figsize=[10,4])
    
    ax.hlines(hist.binsY[slice_row], binRange[0][0], binRange[0][1], color="red", linestyle=":", zorder=10000)
    ax.hlines(hist.binsY[slice_row+1], binRange[0][0], binRange[0][1], color="red", linestyle=":", zorder=10000)
    ax.vlines(binRange[0,0], hist.binsY[slice_row], hist.binsY[slice_row+1], color="red", linestyle=":", zorder=10000)
    ax.vlines(binRange[0,1], hist.binsY[slice_row], hist.binsY[slice_row+1], color="red", linestyle=":", zorder=10000)
    im = hist.draw(ax, extent=binRange.flatten(), cbar_label="Risetime [ns]", overflow=True, vmin=zRange[0], vmax=zRange[1])
    
    ax.set_aspect(25/35)
    ax.grid(False)
    ax.set_xticks([0,1])
    ax.set_yticks([0,1])
    
    axs = fig.get_axes()
    
    Hist.finalize(runs[runID], fig, ax, title="Inpix Risetime\n(MPV)", xlabel="x [pix]", ylabel="y [pix]", param_narrow=True, param_fontsize=6)
    ax.grid(False)
    fig.savefig("output/Inpixel/"+str(runID)+"_MPV.pdf")
    fig.savefig("output/Inpixel/"+str(runID)+"_MPV.png", dpi=400)
    
    fig, ax = Hist.create_fig(1,1, figsize=[5,4])
    
    slice_bins = hist.binsX
    slice_hist = hist.val[1:-1,slice_row+1]
    slice_hist_e = np.sqrt(hist.err[1:-1,slice_row+1])
    slice_binWidth = hist.getBinWidthX()
    slice_xVal = slice_bins[:-1] + slice_binWidth/2
    
    # ax.stairs(slice_hist, slice_bins, fill=True, color="black", alpha=0.5)
    # ax.stairs(slice_hist, slice_bins, fill=False, color="black", alpha=1)
    ax.errorbar(slice_xVal, slice_hist, xerr=slice_binWidth/2 ,yerr=slice_hist_e, color="black", linestyle="", capsize=3, markersize=2)
    ax.set_xlim(hist.getBinRangeX())
    ax.set_ylim(ax.get_ylim()[0], ax.get_ylim()[1]*1.2)
    
    
    Hist.finalize(runs[runID], fig, ax, title="Inpix Risetime\n(MPV)", xlabel="x [pix]", ylabel="Risetime [ns]", param_narrow=True, param_fontsize=6)
    
    Text = "errorbars are binwidth\nin histogram for\nMPV-ing"
    patch = mpl.patches.Patch(color='white', label=Text, alpha=0)
    handles, labels = ax.get_legend_handles_labels()
    handles.append(patch)
    ax.legend(handles=handles, loc="upper left")
    ax.grid(False)
    
    fig.savefig("output/Inpixel/"+str(runID)+"_MPV_slice"+str(slice_row+1)+".pdf")
    fig.savefig("output/Inpixel/"+str(runID)+"_MPV_slice"+str(slice_row+1)+".png", dpi=400)

In [ ]:
# hitmap
runID = 190
binN = [14,10]
cols = [M["TrkCol"], M["TrkRow"]]

mask = runs[runID]["data"][:,:,M["TrkResX"]] < 15
mask = np.logical_and(mask, runs[runID]["data"][:,:,M["TrkResY"]] < 15)
mask = np.logical_and(mask, runs[runID]["data"][:,:,M["PixInCluster"]] == 1)
entries = arr([runs[runID]["data"][:,:,col].flatten()[mask.flatten()] for col in cols])

binRange = [[min(entries[0]), max(entries[0])],[min(entries[1]),max(entries[1])]]
binRange = [[-0.5,1.5],[-0.5,1.5]]

hitmap = Hist_2D(binN, binRange)
hitmap.fill(entries[0], entries[1])

fig, ax = Hist.create_fig(1,1, figsize=[10.,4.])

hitmap.draw(ax, cmap="viridis", extent=[0,35,0,25], overflow=True)

ax.set_xticks([0,1])
ax.set_yticks([0,1])
ax.set_aspect(25/35)

Hist.finalize(runs[runID], fig, ax, title="Hitmap", xlabel="x [pix]", ylabel="y [pix]", param_narrow=True, param_fontsize=6)


In [ ]:
Name = "Falltime"
runID = 190
shareCoords = False
fig, axs = plt.subplots(1, 1, figsize=(4,3), sharex=shareCoords, sharey=shareCoords)

binN = 20
binRange = [0,600]
# binRange = None
column = M[Name]

# mask = np.any(runs[runIDs[0]]["data"][:,:,M["Charge"]] > 2000, axis=1)
pixWithHighestCharge = np.argmax(runs[runIDs[0]]["data"][:,:,M["Charge"]], axis=1)

for i_pix in range(4):
    # mask = runs[runIDs[0]]["data"][:,i_pix,M["Charge"]] >= 220
    # print(mask)
    # mask = i_pix == pixWithHighestCharge
    entries = runs[runIDs[0]]["data"][:,i_pix, column]
    entries = entries[entries != 0]
    hist, bins = np.histogram(entries, bins=binN, range=binRange)
    
    Hist.draw(axs, hist, bins, color="C"+str(i_pix), fill_alpha=0.1, label="Pix "+str(i_pix+1)+" (N={:.1f}k)".format(np.sum(hist)/1000))
    
axs.set_xlim(binRange)
Hist.finalize(runs[runID], fig, axs, Name, "Entries", Name, measurement="TB", logy=False)

In [ ]:
runID = 190

rangeX = [0, 50]
rangeY = [0, 300]
binNX = 20
binNY = 20
NameX = "Risetime"
NameY = "Falltime"
shareCoords = True
logScale = True

colX = M[NameX]
colY = M[NameY]
entriesX = runs[runID]["data"][:,i_pix,colX]
entriesY = runs[runID]["data"][:,i_pix,colY]

pixWithHighestCharge = np.argmax(runs[runIDs[0]]["data"][:,:,M["Charge"]], axis=1)


# hist, binsx, binsy = np.histogram2d(a, b)

# fig, axs = plt.subplots(2,2, figsize=(6, 4))

fig, axs = Hist.create_fig(2,2, [8,6], sharex=shareCoords, sharey=shareCoords)

for i_pix in range(4):
    # mask = i_pix == pixWithHighestCharge
    mask = len(runs[runID]["data"][:,i_pix,colX])*[True]
    # mask = np.logical_and(runs[runID]["data"][mask,i_pix,colX] != 0, runs[runID]["data"][mask,i_pix,colY] != 0) # remove 0 entries
    # mask = runs[runID]["data"][mask,i_pix,colY] != 0 # remove 0 entries
    entriesX = runs[runID]["data"][mask,i_pix,colX]
    entriesY = runs[runID]["data"][mask,i_pix,colY]
    hist, binsx, binsy = np.histogram2d(entriesX, entriesY, bins=[binNX, binNY], range=[rangeX, rangeY])

    extent = [binsx[0], binsx[-1], binsy[0], binsy[-1]]

    if not logScale:
        axs[i_pix].imshow(hist.T, extent=extent, aspect="auto", origin='lower', cmap='viridis')
    else:
        axs[i_pix].imshow(hist.T, extent=extent, aspect="auto", origin='lower', norm=LogNorm(), cmap='viridis')

# lt.imshow(H, interpolation='nearest', origin='lower',

        # extent=[xedges[0], xedges[-1], yedges[0, yedges[-1]])

Hist.finalize(runs[runID], fig, axs, title=NameY+" vs "+NameX, xlabel=NameX, ylabel=NameY)

In [ ]:
%matplotlib inline
# %matplotlib qt

runID = 190
binN = [14,10]
cols = [M["TrkCol"], M["TrkRow"]]

cm = mpl.colormaps["viridis"]

mask = runs[runID]["data"][:,:,M["TrkResX"]] < 15
mask = np.logical_and(mask, runs[runID]["data"][:,:,M["TrkResY"]] < 15)
mask = np.logical_and(mask, runs[runID]["data"][:,:,M["PixInCluster"]] == 1)
entries = arr([runs[runID]["data"][:,:,col].flatten()[mask.flatten()] for col in cols])

binRange = [[min(entries[0]), max(entries[0])],[min(entries[1]),max(entries[1])]]
binRange = [[-0.5,1.5],[-0.5,1.5]]

hist, binsX, binsY = np.histogram2d(entries[0], entries[1], bins=binN, range=binRange)
print(binsX)
print(binsX[:-1]-(binsX[1]-binsX[0])/2)

barEdgesX = binsX[:-1]
barEdgesY = binsY[:-1]
barWidthX = binsX[1]-binsX[0]
barWidthY = binsY[1]-binsY[0]


barEdgesArrayX = arr([barEdgesX for i in range(len(barEdgesY))]).T.flatten()
barEdgesArrayY = arr([barEdgesY for i in range(len(barEdgesX))]).flatten()
barEdgesArrayZ = np.zeros_like(hist).flatten()
barWidthArrayX = np.ones_like(hist).flatten()*barWidthX
barWidthArrayY = np.ones_like(hist).flatten()*barWidthY
barWidthArrayZ = hist.flatten()

posArrayX = barEdgesArrayX + barWidthArrayX/2
posArrayY = barEdgesArrayY + barWidthArrayY/2
posArrayZ = hist.flatten()
step = 4
posArrayX = posArrayX[::step]
posArrayY = posArrayY[::step]
posArrayZ = posArrayZ[::step]


fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111, projection="3d")

ax.bar3d(barEdgesArrayX, barEdgesArrayY, barEdgesArrayZ, barWidthArrayX, barWidthArrayY, barWidthArrayZ, color=cm(hist.flatten()/np.max(hist.flatten())), zsort='average', alpha=.6)
# ax.plot_wireframe(posArrayX.reshape(hist.shape), posArrayY.reshape(hist.shape), hist, color="black", alpha=0.7)
ax.errorbar(posArrayX, posArrayY, posArrayZ, zerr=np.sqrt(posArrayZ.flatten()), fmt=".", color="black", capsize=2, markersize=1, elinewidth=1, zorder=1000000, barsabove=True)

if False:
    # xpos, ypos = np.meshgrid(xedges[:-1]+xedges[1:], yedges[:-1]+yedges[1:])

    print(xpos)

    x_centers = arr([(xedges[:-1] + xedges[1:]) / 2 for i in range(len(yedges)-1)]).T
    y_centers = arr([(yedges[:-1] + yedges[1:]) / 2 for i in range(len(xedges)-1)])
    # coords = np.array([x_centers, y_centers])

    # x_centers = xpos.T
    # y_centers = ypos.T

    xpos = xpos.flatten()/2.
    ypos = ypos.flatten()/2.
    zpos = np.zeros_like (xpos)

    dx = xedges [1] - xedges [0]
    dy = yedges [1] - yedges [0]
    dz = hist.flatten()

    # x_centers -= dx/2
    # y_centers -= dy/2

    # x_centers = arr([xpos + dx/2 for i in range(len(ypos))])
    # y_centers = arr([ypos + dy/2 for i in range(len(xpos))])
    max_height = np.max(dz)   # get range of colorbars so we can normalize
    min_height = np.min(dz)
    # scale each z to [0,1], and get their rgb values
    rgba = [cm((k-min_height)/max_height) for k in dz] 

    fig = plt.figure(figsize=(10,6))
    ax = fig.add_subplot(111, projection="3d")

    # ax.errorbar(x_centers.flatten(), y_centers.flatten(), hist.flatten(), zerr=np.sqrt(hist.flatten()), fmt="none", ecolor="black", capsize=2)
    # ax.bar3d(xpos, ypos, zpos, dx, dy, dz, color=rgba, zsort='average', alpha=0.7)
    print(x_centers.shape, y_centers.shape, hist.shape)
    ax.plot_wireframe(x_centers, y_centers, hist, color="black", alpha=0.7)
    ax.errorbar(xpos+dx/2, ypos+dy/2, dz, zerr=np.sqrt(hist.flatten()), fmt="None", ecolor="black", capsize=2)

    # ax.set_aspect(25/35)\
    
if True:
    ax.set_box_aspect([35,25,30])
    plt.title("ER1 Hitmap")
    plt.xlabel("x [pix]", loc="right")
    plt.ylabel("y [pix]", loc="top")
    plt.savefig("Your_title_goes_here")
    plt.show()
    # plt.clf()
    